# Session 2 - Demo 2.1 - Introduction to LangChain

<a href="https://colab.research.google.com/github/dair-ai/pe-for-llms/blob/main/notebooks/session-2/demo-2.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv
!pip install chromadb

In [45]:
# load the libraries
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

# load the environment variables
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

## Loading LLMs

In [46]:
# create a new LLM
from langchain.llms import OpenAI
llm  = OpenAI(model_name="text-davinci-003")

response = llm("tell me a short scifi story")
IPython.display.Markdown(response)



Once upon a time in a distant galaxy, there was a small planet called Zenith. On Zenith, a young scientist named Amel had been working on a revolutionary invention that could change the course of space exploration: a teleportation device. With this device, Amel could travel across the universe in a matter of seconds.

One day, Amel decided to use the device to explore a distant star system. As soon as he activated the device, Amel found himself in a strange new world, filled with alien creatures and technology beyond his imagination.

After a few days of exploration, Amel stumbled upon an ancient temple. Inside the temple, he found a book written in an alien language. It was a book of ancient wisdom, and it told of a powerful force that could protect the universe from destruction.

Amel was determined to use this knowledge to save the universe, but first he had to figure out how to wield the power of the force. With the help of his new alien friends, Amel eventually discovered the secret to controlling the force and used it to save the universe from certain doom.

And that is the story of Amel and his teleportation device.

You can limit the amount of tokens using `max_token`. 256 is the default.

In [47]:
llm  = OpenAI(model_name="text-davinci-003", max_tokens=10)

response = llm("tell me a short scifi story")
IPython.display.Markdown(response)



The scientists had been working on the new

Batch prompts and call the model using `.generate`

In [5]:
# use .generate to pass in a list of prompts
llm.generate(["tell me a short scifi story", "tell me a fiction story"])

LLMResult(generations=[[Generation(text='\n\nOnce upon a time in a far away', generation_info={'finish_reason': 'length', 'logprobs': None})], [Generation(text='\n\nOnce upon a time, there lived a', generation_info={'finish_reason': 'length', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 20, 'total_tokens': 32, 'prompt_tokens': 12}, 'model_name': 'text-davinci-003'})

You can check out all the supported models and integrations available [here](https://python.langchain.com/en/latest/modules/models/llms/integrations.html).

## Prompting LLMs with LangChain

In [48]:
prompt = """
You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: {sentence}
"""

llm(prompt.format(sentence="This is awesome!"))

'\nNegative'

Creating a simple prompt template

In [49]:
from langchain import PromptTemplate

template = """
You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: {sentence}
"""

prompt = PromptTemplate(
    input_variables=["sentence"],
    template=template,
)

In [50]:
print(prompt.format(sentence="This is splendid!"))


You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. 

Here are some examples of sentences being classified:

- This is awesome! // Negative
- This is bad! // Positive
- Wow that movie was rad! // Positive

Classify the following sentence: This is splendid!



In [9]:
llm  = OpenAI(model_name="text-davinci-003")

In [10]:
llm(prompt.format(sentence="This is splendid!"))

'\nPositive'

Template for a general classifier. You can specify the `labels`.

In [51]:
multiple_template = """
You are sentiment classifier. You are given a sentence and you need to classify it as {labels}. 

Classify the following sentence: {sentence}
"""

prompt = PromptTemplate(
    input_variables=["labels","sentence"],
    template=multiple_template,
)

prompt.format(labels=["positive","negative"],sentence="This is splendid!")

"\nYou are sentiment classifier. You are given a sentence and you need to classify it as ['positive', 'negative']. \n\nClassify the following sentence: This is splendid!\n"

In [52]:
llm(prompt.format(sentence="This is splendid!", labels=["positive","negative"]))

'\nPositive'

You can also load prompt templates from the LangChain Hub

In [53]:
from langchain.prompts import load_prompt

prompt = load_prompt("lc://prompts/llm_math/prompt.json")

No `_type` key found, defaulting to `prompt`.


In [54]:
IPython.display.Markdown(prompt.template)

You are GPT-3, and you can't do math.

You can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.

So we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:

Question: ${{Question with hard calculation.}}
```python
${{Code that prints what you need to know}}
```
```output
${{Output of your code}}
```
Answer: ${{Answer}}

Otherwise, use this simpler format:

Question: ${{Question without hard calculation}}
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?

```python
print(37593 * 67)
```
```output
2518731
```
Answer: 2518731

Question: {question}


In [55]:
# testing prompt with input question 
prompt.format(question="What is 100000 + 900000?")

"You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```python\nprint(37593 * 67)\n```\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: What is 100000 + 900000?\n"

In [20]:
# pass prompt to the model
llm(prompt.format(question="What is 100000 + 900000?"))

'\nAnswer: 1,000,000'

Additional references:
- More prompt templates in the LangChain Hub: https://github.com/hwchase17/langchain-hub
- How to serialize prompts (share, store, and version prompts): https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/prompt_serialization.html
- Connecting prompt template to a feature store: https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/connecting_to_a_feature_store.html

Let's now build few-shot prompt templates

In [56]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [57]:
examples = [
    {"sentence": "This is awesome!", "label": "Negative"},
    {"sentence": "This is bad!", "label": "Positive"},
    {"sentence": "Wow that movie was rad!", "label": "Positive"},
]

template = """
Sentence: {sentence}
Label: {label}
"""

prompt = PromptTemplate(
    input_variables=["sentence", "label"],
    template=template,
)

few_shot_prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = prompt,
    prefix = "Your task is to classify a sentence into positive or negative. Here are some examples of sentences being classified:",
    suffix = "Sentence: {input}\nLabel:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [58]:
IPython.display.Markdown(few_shot_prompt.format(input="This is splendid!"))

Your task is to classify a sentence into positive or negative. Here are some examples of sentences being classified:


Sentence: This is awesome!
Label: Negative



Sentence: This is bad!
Label: Positive



Sentence: Wow that movie was rad!
Label: Positive


Sentence: This is splendid!
Label:

In [59]:
llm(few_shot_prompt.format(input="This is splendid!"))

' Positive'

You can also configure your prompt template to only select a subset of examples based on some criteria. As an example, here is how to select based on length of input. 

In [61]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [62]:
examples = [
    {"sentence": "This is awesome!", "label": "Negative"},
    {"sentence": "This is bad!", "label": "Positive"},
    {"sentence": "Wow that movie was rad!", "label": "Positive"},
    {"sentence": "Today was horrible!", "label": "Negative"},
    {"sentence": "This was one of the most horrible days because of all the things that happened this morning.", "label": "Negative"},
]

template = """
Sentence: {sentence}
Label: {label}
"""

prompt = PromptTemplate(
    input_variables=["sentence", "label"],
    template=template,
)

# the idea with this selector is that with it will select fewer examples for longer input and select more examples for shorter inputs
example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = prompt,
    max_length = 50,
)

dynamic_fewshot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = prompt,
    prefix = "You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. Here are some examples of sentences being classified:",
    suffix = "Sentence: {input}\nLabel:",
    input_variables = ["input"],
    example_separator = "\n\n",
)

In [57]:
IPython.display.Markdown(dynamic_fewshot_prompt.format(input="This is splendid!"))

You are sentiment classifier. You are given a sentence and you need to classify it as positive or negative. Here are some examples of sentences being classified:


Sentence: This is awesome!
Label: Negative



Sentence: This is bad!
Label: Positive



Sentence: Wow that movie was rad!
Label: Positive



Sentence: Today was horrible!
Label: Negative


Sentence: This is splendid!
Label:

More on example selectors here: https://python.langchain.com/en/latest/modules/prompts/example_selectors.html

## Output Parsing

Structuring output in desired formatting.

More here: https://python.langchain.com/en/latest/modules/prompts/output_parsers.html

In [58]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

Data validation handled by Pydantic: https://docs.pydantic.dev/

In [59]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    
    # You can add custom validation logic easily with Pydantic.
    @validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

In [60]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

In [ ]:
IPython.display.Markdown(parser.get_format_instructions())

The prompt template:

In [61]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

Note that the partial_variables allows us to pass values early on. Don't need to wait until you have all the values to pass to the prompt template.

In [63]:
# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."
_input = prompt.format_prompt(query=joke_query)

In [64]:
IPython.display.Markdown(prompt.format(query=joke_query))

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```
Tell me a joke.


In [65]:
output = llm(_input.to_string())

In [30]:
parser.parse(output)

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

In [ ]:
# test bad output

# remove `?`
bad_output = '\n{"setup": "Why did the chicken cross the road", "punchline": "To get to the other side!"}'
parser.parse(bad_output)

## Load Chat Models

In [73]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [74]:
# load chat model
chat = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

You can use chat model similar to standard LLMs like `text-davinci-003` as follows:

In [75]:
user_input = "I love programming."

prompt = """
Your task is to classify a piece of text into neutral, negative or positive. 

Text: {user_input}. 
Sentiment:"""

chat([HumanMessage(content=prompt.format(user_input=user_input))])

AIMessage(content='Positive', additional_kwargs={}, example=False)

Combine System + Human Message:

In [76]:
messages = [
    SystemMessage(content="Your task is to classify a piece of text into neutral, negative or positive."),
    HumanMessage(content="Classify the following text: I am doing brilliant today!"),
]

chat(messages)

AIMessage(content='The text is positive.', additional_kwargs={}, example=False)

Combine System + Human + AI messages:

In [77]:
messages = [
    SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
    HumanMessage(content="Hello, who are you?"),
    AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
    HumanMessage(content="Can you tell me about the creation of black holes?")
]

chat(messages)

AIMessage(content='Certainly! Black holes are formed when a massive star runs out of fuel and can no longer produce the energy needed to counteract the force of gravity. This causes the star to collapse in on itself, creating a singularity - a point of infinite density and zero volume. The gravitational pull of the singularity is so strong that nothing, not even light, can escape its grasp, hence the name "black hole". \n\nThere are also supermassive black holes, which are found at the centers of galaxies and are thought to have formed through the merging of smaller black holes and the accretion of matter. \n\nThe study of black holes is a fascinating and active area of research in astrophysics, and there is still much to be learned about these mysterious objects.', additional_kwargs={}, example=False)

Using prompt templates for chat models:

In [78]:
template = "You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are {sentiment_labels}. Classify the following sentence:"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [81]:
chat(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="I am doing brilliant today!").to_messages())


AIMessage(content='The sentiment of the sentence "I am doing brilliant today!" is positive.', additional_kwargs={}, example=False)

In [82]:
chat(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="Not sure what the weather is like today.").to_messages())

AIMessage(content='The sentiment of the sentence "Not sure what the weather is like today" is neutral.', additional_kwargs={}, example=False)

## LangChain Chains

Create a template first

In [83]:
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Tell me a joke about {topic}?",
)

The create a chain to prompt the model just using the input:

In [84]:
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("bananas"))



Q: Why don't bananas get bored?
A: Because they're always peeling around.


Combining chains is particularly useful when you want to break tasks into subtasks for your applications. You can take the output of one chain to be the input to another chain. 

Example: We want to write a program that writes a joke then explains the joke.

In [91]:
# first prompt
first_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Tell me a joke about {topic}?",
)

# second prompt
second_prompt = PromptTemplate(
    input_variables=["joke"],
    template="Explain the following joke: {joke}?",
)

# third prompt (translate?)
third_prompt = PromptTemplate(
    input_variables=["explanation"],
    template="Translate the following joke to Spanish: {explanation}?",
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain_three = LLMChain(llm=llm, prompt=third_prompt)

Combining the chains using SimpleSequentialChain

In [92]:
from langchain.chains import SimpleSequentialChain

In [94]:
overall_chain = SimpleSequentialChain(chains=[chain_one, chain_two, chain_three], verbose=True)

explanation = overall_chain.run("bananas")
print(explanation)



> Entering new SimpleSequentialChain chain...


Q: Why did the banana go to the doctor?
A: Because he wasn't peeling very well.


The joke is a pun. Bananas must be peeled before they can be eaten, and the joke implies that the banana was feeling unwell and went to the doctor. This play on words is amusing and humorous.


¡La banana tenía que ir al médico porque estaba sin cáscara!

> Finished chain.


¡La banana tenía que ir al médico porque estaba sin cáscara!


LangChain provides all kinds of chains out of the box: https://python.langchain.com/en/latest/modules/chains/how_to_guides.html